## This notebook contains various information and trial queries of the MAST database API, tailored towards the needs of the JWQL application.

#### Available `services`

- `Mast.Jwst.Filtered.Miri`
- `Mast.Jwst.Filtered.Nircam`
- `Mast.Jwst.Filtered.Niriss`
- `Mast.Jwst.Filtered.Nirspec`
- `Mast.Jwst.Filtered.Fgs`

In [1]:
import glob
import os
from astropy.io import fits
from astroquery.mast import Mast

#### A very basic example of how to submit a query

In [2]:
service = "Mast.Jwst.Filtered.Miri"
params = {"columns":"*","filters":[]}
response = Mast.service_request_async(service,params)
result = response[0].json()

#### `response` returns a list of one item of type `requests.models.Response` which can easily be converted to `json`/`dict`

In [3]:
type(result)

dict

In [4]:
result.keys()

dict_keys(['status', 'data', 'fields', 'paging', 'msg'])

#### The `fields` key lists the resulting column names and their data type

In [ ]:
result['fields']

#### The `data` key contains the values for each row returned from the query

In [ ]:
result['data']

In [7]:
len(result['data'])

495

#### Specific columns can be pulled out by iterating over `data`

In [ ]:
filenames = [item['filename'] for item in result['data']]
print(filenames)

In [9]:
print(len(filenames))
print(len(set(filenames)))

495
495


#### The `status` key appears to return the status of the query

In [10]:
result['status']

'COMPLETE'

#### The `paging` key describes the amount of data returned by the query

In [11]:
result['paging']

{'page': 1,
 'pageSize': 50000,
 'pagesFiltered': 1,
 'rows': 495,
 'rowsFiltered': 495,
 'rowsTotal': 495}

#### I'm not sure what the `msg` key stores

In [12]:
result['msg']

''

#### Testing the services for the other JWST instruments

In [13]:
services = ['Mast.Jwst.Filtered.Nircam', 'Mast.Jwst.Filtered.Nirspec',
            'Mast.Jwst.Filtered.Niriss', 'Mast.Jwst.Filtered.Miri', 
            'Mast.Jwst.Filtered.Fgs']
for service in services:
    params = {"columns":"*","filters":[]}
    response = Mast.service_request_async(service,params)
    result = response[0].json()
    print('{}: {} rows'.format(service, len(result['data'])))

Mast.Jwst.Filtered.Nircam: 2748 rows
Mast.Jwst.Filtered.Nirspec: 878 rows
Mast.Jwst.Filtered.Niriss: 387 rows
Mast.Jwst.Filtered.Miri: 495 rows
Mast.Jwst.Filtered.Fgs: 152 rows


### Example Query: Find all MIRI data taken between two time intervals

In [14]:
service = "Mast.Jwst.Filtered.Miri"
params = {"columns":"filename, expstart","filters":[
                         {"paramName":"expstart",
                          "values":[{"min":57404.04, "max":57404.07}],
                         }]}
response = Mast.service_request_async(service,params)
result = response[0].json()

In [15]:
result['data']

[{'expstart': 57404.04476711806,
  'filename': 'jw80600003001_02101_00001_mirimage_cal.fits'},
 {'expstart': 57404.04476711806,
  'filename': 'jw80600003001_02101_00001_mirimage_i2d.fits'},
 {'expstart': 57404.05132637732,
  'filename': 'jw80600004001_02101_00001_mirimage_cal.fits'},
 {'expstart': 57404.05132637732,
  'filename': 'jw80600004001_02101_00001_mirimage_i2d.fits'},
 {'expstart': 57404.04108451065,
  'filename': 'jw80600001001_02103_00001_mirimage_uncal.fits'},
 {'expstart': 57404.04476711806,
  'filename': 'jw80600003001_02101_00001_mirimage_rate.fits'},
 {'expstart': 57404.04476711806,
  'filename': 'jw80600003001_02101_00001_mirimage_rateints.fits'},
 {'expstart': 57404.05979692546,
  'filename': 'jw80600005001_02101_00001_mirimage_uncal.fits'},
 {'expstart': 57404.05132637732,
  'filename': 'jw80600004001_02101_00001_mirimage_rate.fits'}]

#### Note that this doesn't seem to be possible with `date_obs`

In [16]:
service = "Mast.Jwst.Filtered.Miri"
params = {"columns":"filename, date_obs","filters":[
                         {"paramName":"date_obs",
                          "values":[{"min":'01-17-2016', "max":'01-18-2016'}],
                         }]}
response = Mast.service_request_async(service,params)
result = response[0].json()

In [17]:
len(result['data'])

495

In [ ]:
result['data']

### Example Query: What data are currently proprietary?

In [19]:
# Not implemented

### More example queries

In [20]:
# Not implemented

## Determining which header keywords exist in file headers, but not the MAST database

In [21]:
test_filename = 'jw00329003001_02101_00001_nrca2_rate.fits' # Note the full path is excluded for security reasons

In [22]:
# Get set of all header keywords that exist in PRIMARY and SCI extension of files
header0_keys_fits = set(list(fits.getheader(test_filename, 0).keys()))
header0_keys_fits = set([item.lower().replace('-', '_') for item in header0_keys_fits])
header1_keys_fits = set(list(fits.getheader(test_filename, 1).keys()))
header1_keys_fits = set([item.lower().replace('-', '_') for item in header1_keys_fits])
header_keys_fits = header0_keys_fits | header1_keys_fits

In [23]:
# Get set of all keywords available in the MAST API
service = "Mast.Jwst.Filtered.Nircam"
params = {"columns":"*","filters":[{"paramName":"filename",
                          "values":[os.path.basename(test_filename)]}]}
response = Mast.service_request_async(service,params)
result = response[0].json()
header_keys_db = set(list(result['data'][0].keys()))

In [24]:
# Keywords that are in the MAST API but not the FITS headers
header_keys_db - header_keys_fits

{'ArchiveFileID',
 'FileSetId',
 'FileTypeID',
 'asnpool',
 'asntable',
 'bkglevel',
 'bkgsub',
 'checksum',
 'dataURI',
 'fileSetName',
 'fileSize',
 'ingestCompletionDate',
 'ingestStartDate',
 'isItar',
 'isRestricted',
 'isStale',
 'nwfsest',
 'productLevel',
 'psfref',
 'publicReleaseDate',
 'pwfseet',
 'selfref',
 'srctype'}

In [25]:
# Keywords that are in the FITS headers but not the MAST API
header_keys_fits - header_keys_db

{'',
 'bitpix',
 'bunit',
 'cal_vcs',
 'cal_ver',
 'cdelt1',
 'cdelt2',
 'comment',
 'crds_ctx',
 'crds_ver',
 'crpix1',
 'crpix2',
 'crval1',
 'crval2',
 'ctype1',
 'ctype2',
 'cunit1',
 'cunit2',
 'datamodl',
 'date_end',
 'dec_ref',
 'dec_v1',
 'extend',
 'extname',
 'extver',
 'gcount',
 'naxis',
 'naxis1',
 'naxis2',
 'pa_aper',
 'pa_v3',
 'pc1_1',
 'pc1_2',
 'pc2_1',
 'pc2_2',
 'pcount',
 'r_dark',
 'r_gain',
 'r_linear',
 'r_mask',
 'r_persat',
 'r_readno',
 'r_satura',
 'r_superb',
 'r_trpden',
 'r_trppar',
 'ra_ref',
 'ra_v1',
 'radesys',
 'roll_ref',
 's_dark',
 's_dqinit',
 's_ganscl',
 's_grpscl',
 's_jump',
 's_linear',
 's_persis',
 's_ramp',
 's_refpix',
 's_satura',
 's_superb',
 'simple',
 'time_end',
 'time_obs',
 'v2_ref',
 'v3_ref',
 'v3i_yang',
 'va_scale',
 'vparity',
 'wcsaxes',
 'xtension'}